In [1]:
!pip install transformers datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.6 MB/s eta 0:00:00
Looking in index

In [2]:
from google.colab import drive
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import IPython
from datasets import load_dataset
import datasets
import evaluate

from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy

##Preprocessing

In [3]:
def tokenize_mnli(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

def tokenize_wnli(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

def tokenize_qnli(example):
    return tokenizer(example["question"], example["sentence"], truncation=True)

In [ ]:
# raw_datasets = load_dataset("snli")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

##Datasets

In [4]:
mnli_dataset = load_dataset("glue", "mnli")
wnli_dataset = load_dataset("glue", "wnli")
qnli_dataset = load_dataset("glue", "qnli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
df_mnli_train = pd.DataFrame(mnli_dataset['train'].select(random.sample(range(0, 392702), 635)))
df_qnli_train = pd.DataFrame(qnli_dataset['train'].select(random.sample(range(0, 104743), 635)))

df_mnli_val = pd.DataFrame(mnli_dataset['validation_mismatched'].select(random.sample(range(0, 9832), 71)))
df_qnli_val = pd.DataFrame(qnli_dataset['validation'].select(random.sample(range(0, 5463), 71)))

df_mnli_test = pd.DataFrame(mnli_dataset['test_mismatched'].select(random.sample(range(0, 9847), 146)))
df_qnli_test = pd.DataFrame(qnli_dataset['test'].select(random.sample(range(0, 5463), 146)))

In [15]:
df_wnli_test = pd.DataFrame(wnli_dataset['train'])

In [16]:
df_wnli_test.label.unique()

array([1, 0])

In [10]:
df_qnli_train.label.unique()

array([0, 1])

In [17]:
df_mnli_train.label.unique()

array([1, 0, 2])

In [ ]:
# convert to Dataset type

mnli_dataset = datasets.DatasetDict(
    {"train": Dataset.from_pandas(df_mnli_train),
     "validation": Dataset.from_pandas(df_mnli_val),
     "test": Dataset.from_pandas(df_mnli_test)
    })

qnli_dataset = datasets.DatasetDict(
    {"train": Dataset.from_pandas(df_qnli_train),
     "validation": Dataset.from_pandas(df_qnli_val),
     "test": Dataset.from_pandas(df_qnli_test)
    })

In [13]:
wnli_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 635
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 71
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 146
    })
})

## MNLI

In [ ]:
tokenized_datasets = mnli_dataset.map(tokenize_mnli, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

mnli_tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

mnli_tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

mnli_tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    mnli_tf_train_dataset,
    # validation_data=mnli_tf_validation_dataset,
    epochs=5
)

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
80/80 [==============================] - 32s 134ms/step - loss: 1.1271 - accuracy: 0.2976
Epoch 2/5
80/80 [==============================] - 11s 132ms/step - loss: 1.0660 - accuracy: 0.4157
Epoch 3/5
80/80 [==============================] - 11s 134ms/step - loss: 0.9611 - accuracy: 0.5638
Epoch 4/5
80/80 [==============================] - 11s 138ms/step - loss: 0.7305 - accuracy: 0.7339
Epoch 5/5
80/80 [==============================] - 11s 140ms/step - loss: 0.4564 - accuracy: 0.8898


In [ ]:
model.evaluate(mnli_tf_validation_dataset)

9/9 [==============================] - 4s 63ms/step - loss: 1.1538 - accuracy: 0.5775


[1.1538265943527222, 0.577464759349823]

In [ ]:
model.evaluate(wnli_tf_validation_dataset)

9/9 [==============================] - 1s 63ms/step - loss: 1.5507 - accuracy: 0.5634


[1.5506738424301147, 0.5633803009986877]

In [ ]:
model.evaluate(qnli_tf_validation_dataset)

9/9 [==============================] - 1s 56ms/step - loss: 0.9448 - accuracy: 0.5493


[0.9448274374008179, 0.5492957830429077]

## WNLI

In [ ]:
# wnli_dataset = load_dataset("glue", "wnli")
wnli_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 635
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 71
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 146
    })
})

In [ ]:
tokenized_datasets = wnli_dataset.map(tokenize_wnli, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

wnli_tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

wnli_tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

wnli_tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:


model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    wnli_tf_train_dataset,
    # validation_data=wnli_tf_validation_dataset,
    epochs=5
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
80/80 [==============================] - 27s 136ms/step - loss: 0.8009 - accuracy: 0.4661
Epoch 2/5
80/80 [==============================] - 11s 136ms/step - loss: 0.7176 - accuracy: 0.5118
Epoch 3/5
80/80 [==============================] - 11s 137ms/step - loss: 0.7156 - accuracy: 0.4583
Epoch 4/5
80/80 [==============================] - 11s 132ms/step - loss: 0.7185 - accuracy: 0.4567
Epoch 5/5
80/80 [==============================] - 11s 136ms/step - loss: 0.7027 - accuracy: 0.5039


In [ ]:
model.evaluate(wnli_tf_validation_dataset)

9/9 [==============================] - 3s 60ms/step - loss: 0.7220 - accuracy: 0.4225


[0.7219671010971069, 0.4225352108478546]

In [ ]:
model.evaluate(mnli_tf_validation_dataset)

9/9 [==============================] - 0s 50ms/step - loss: 1.9239 - accuracy: 0.2676


[1.9238758087158203, 0.26760563254356384]

In [ ]:
model.evaluate(qnli_tf_validation_dataset)

9/9 [==============================] - 1s 53ms/step - loss: 0.7009 - accuracy: 0.4789


[0.7009159922599792, 0.47887325286865234]

## QNLI

In [ ]:
# qnli_dataset = load_dataset("glue", "qnli")
qnli_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 635
    })
    validation: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 71
    })
    test: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 146
    })
})

In [ ]:
tokenized_datasets = qnli_dataset.map(tokenize_qnli, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

qnli_tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

qnli_tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

qnli_tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    qnli_tf_train_dataset,
    # validation_data=qnli_tf_validation_dataset,
    epochs=5
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
80/80 [==============================] - 28s 157ms/step - loss: 0.9759 - accuracy: 0.4157
Epoch 2/5
80/80 [==============================] - 12s 156ms/step - loss: 0.6043 - accuracy: 0.7591
Epoch 3/5
80/80 [==============================] - 13s 159ms/step - loss: 0.3679 - accuracy: 0.8772
Epoch 4/5
80/80 [==============================] - 13s 158ms/step - loss: 0.1974 - accuracy: 0.9402
Epoch 5/5
80/80 [==============================] - 13s 158ms/step - loss: 0.0758 - accuracy: 0.9906


In [ ]:
model.evaluate(qnli_tf_validation_dataset)

9/9 [==============================] - 3s 64ms/step - loss: 0.6799 - accuracy: 0.8310


[0.6798612475395203, 0.8309859037399292]

In [ ]:
model.evaluate(mnli_tf_validation_dataset)

9/9 [==============================] - 1s 76ms/step - loss: 2.0719 - accuracy: 0.4366


[2.071913242340088, 0.43661972880363464]

In [ ]:
model.evaluate(wnli_tf_validation_dataset)

9/9 [==============================] - 1s 74ms/step - loss: 1.5662 - accuracy: 0.5634


[1.5662450790405273, 0.5633803009986877]